In [ ]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

from os import path
from keras.utils import image_dataset_from_directory
from matplotlib import pyplot as plt
from keras import layers, losses, optimizers, metrics, callbacks

In [ ]:
tf.config.list_logical_devices()

In [ ]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 32

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
train_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'train.csv')).drop(["Unnamed: 0"], axis=1)
train_labels_df

In [ ]:
dataset_train = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'train'),
    labels=list(train_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_train = dataset_train.map(lambda x, y : (x, tf.one_hot(y, N_CLASS)))

In [ ]:
val_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'val.csv')).drop(["Unnamed: 0"], axis=1)
val_labels_df

In [ ]:
dataset_val = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'val'),
    labels=list(val_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_val = dataset_val.map(lambda x, y: (x, tf.one_hot(y, N_CLASS)))

In [ ]:
test_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'test.csv')).drop(["Unnamed: 0"], axis=1)

dataset_test = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'test'),
    labels=list(test_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_test = dataset_test.map(lambda x, y: (x, tf.one_hot(y, N_CLASS)))

In [ ]:
X, y = next(iter(dataset_train))
X.shape, y.shape

In [ ]:
plt.imshow(X[3].numpy().astype(int))
plt.xlabel(train_labels_df.codes.cat.categories[np.argwhere(y[3].numpy()).flatten()])

In [ ]:
y[3]

## Simple keras sequential model

In [ ]:
def create_model():
    model = keras.Sequential()

    model.add(keras.Input(shape=(32, 32, 3)))

    model.add(layers.Conv2D(32, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.Conv2D(32, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.Conv2D(64, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.Conv2D(64, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.MaxPooling2D(2))

    # Fully connected classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dense(N_CLASS, activation="softmax"))
    return model

model = create_model()
model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
)
checkpoint = callbacks.ModelCheckpoint(
    filepath="simple_cnn.h5",
    monitor="val_categorical_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [ ]:
history = model.fit(
    dataset_train,
    epochs=25,
    workers=2,
    use_multiprocessing=True,
    validation_data=dataset_val,
    shuffle=True,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

## Experiments

Training will be repeated 10 times with different weights initialization.

In [ ]:
TRAINING_SEEDS = list(range(10))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = create_model()

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
    )

    history = model.fit(
        dataset_train,
        epochs=25,
        workers=2,
        use_multiprocessing=True,
        validation_data=dataset_val,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )
    eval_results = model.evaluate(dataset_test)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('simple_cnn_results.csv')